# IN PROGRESS ...

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime

In [2]:
url_main = 'https://www.foxsports.com/soccer/2022-fifa-world-cup/teams'
response = requests.get(url_main)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
country_name = [x.text for x in soup.find_all('h3')]
countries = [x['href'] for x in soup.find_all('a',{'class':'entity-list-row-container image-logo'})]

In [4]:
# running time: 1m 20s

rows=[]
for idx,country in enumerate(countries):

    url = f'https://www.foxsports.com{country}-roster'
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    table = soup.find('div',{'view':'team'})
    title = [x.text.strip() for x in table.find_all('tr')[0]]
    
    for group in table.find_all('tbody')[:-1]:
        for player in group:
            row={}
            row['Country'] = country_name[idx].capitalize()
            row['Name'] = player.find('h3').text
            row[title[1]] = player.find('td',{'data-index':'1'}).text.strip()
            row[title[2]] = player.find('td',{'data-index':'2'}).text.strip()
            row[title[3]] = player.find('td',{'data-index':'3'}).text.strip()
            row[title[4]] = player.find('td',{'data-index':'4'}).text.strip()
            rows.append(row)

In [5]:
data = pd.DataFrame(rows)

date = datetime.now().strftime("%d-%m-%Y %H%M%S") # get local time as string

filePath = f"../ESPN - Output/{url_main.split('/')[-2]} {date} - Webscrape.csv"

data.to_csv(filePath, index=False) # save to file path

data

,Country,Name,POS,AGE,HT,WT
0,Argentina,Esteban Andrada,G,31,"6'4""",182 lbs
1,Argentina,Franco Armani,G,35,"6'2""",194 lbs
2,Argentina,Federico Gomes Gerth,G,18,-,-
3,Argentina,Jeremias Ledesma,G,29,"6'0""",182 lbs
4,Argentina,Agustin Marchesin,G,34,"6'2""",182 lbs
...,...,...,...,...,...,...
1703,Wales,Rabbi Matondo,F,21,"5'8""",145 lbs
1704,Wales,Kieffer Moore,F,29,"6'4""",182 lbs
1705,Wales,Tyler Roberts,F,23,"5'10""",165 lbs
1706,Wales,Sorba Thomas,F,22,"6'0""",-


In [6]:
# Clean table

# TO DO:
# drop duplicated players within a country (ex: Argentina 'Agustin Rossi')

# build dataframe
data = pd.DataFrame(rows)

# drop rows with missing data
data = data[~data.isin(['-']).any(axis=1)]

# replace possition values
POS_values = {x.text.strip().capitalize() for x in table.find_all('th',{'data-index':'0'})}
POS_keys = set(data[title[1]].to_list())
POS_mapped = dict(zip(POS_keys, POS_values))
data.replace({title[1]: POS_mapped}, inplace=True)

# age column to number
data[title[2]] = pd.to_numeric(data[title[2]])

# hight column to number
data[title[3]] = data[title[3]].apply(lambda x: (int(x.split('\'')[0])*12 + int(x.split('\'')[1].replace('\"',''))) * 2.54) # inches to centimeters

# weight column to number
data[title[4]] = data[title[4]].apply(lambda x: round(int(x.split(' ')[0]) / 2.205,1)) # lbs to kg

data


,Country,Name,POS,AGE,HT,WT
0,Argentina,Esteban Andrada,Midfielder,31,193.04,82.5
1,Argentina,Franco Armani,Midfielder,35,187.96,88.0
3,Argentina,Jeremias Ledesma,Midfielder,29,182.88,82.5
4,Argentina,Agustin Marchesin,Midfielder,34,187.96,82.5
5,Argentina,Emiliano Martinez,Midfielder,29,193.04,88.9
...,...,...,...,...,...,...
1701,Wales,Wes Burns,Defender,27,172.72,67.6
1703,Wales,Rabbi Matondo,Defender,21,172.72,65.8
1704,Wales,Kieffer Moore,Defender,29,193.04,82.5
1705,Wales,Tyler Roberts,Defender,23,177.80,74.8


In [7]:
date = datetime.now().strftime("%d-%m-%Y %H%M%S") # get local time as string

filePath = f"../ESPN - Output/{url_main.split('/')[-2]} CLEAN {date} - Webscrape.csv"

data.to_csv(filePath, index=False) # save to file path

In [8]:
# Average per possition
data.loc[:, data.columns != data.columns[0]].groupby('POS').mean().round(1)

,AGE,HT,WT
POS,,,
Defender,27.1,179.0,74.8
Forward,26.7,177.2,71.5
Midfielder,29.7,187.9,82.6
Player count,27.4,181.7,76.1
